In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
from scipy import stats
from sklearn.externals.six import StringIO
from sklearn import preprocessing
from sklearn import cluster, tree, decomposition
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sales=pd.read_excel("C:\\Users\\Ganesh\\Desktop\\Sarap.xlsx")

In [3]:
sales['Ccode']=sales.Ccode.astype(object)

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6766 entries, 0 to 6765
Data columns (total 8 columns):
InvoiceNo             6766 non-null int64
InvoiceDate           6766 non-null object
states                6766 non-null object
Products              6766 non-null object
Ccode                 6766 non-null object
HSNCode               6766 non-null int64
ProductDescription    6766 non-null object
Quantity              6766 non-null int64
dtypes: int64(3), object(5)
memory usage: 423.0+ KB


In [7]:
customer_item_matrix=sales.pivot_table(index="Ccode",columns="HSNCode",values="Quantity", aggfunc='sum')

In [8]:
customer_item_matrix.head()

HSNCode,9109100,13019013,19059040,20019000,20049000,20079990,20093900,21033000,21039090,21069099,22090020
Ccode,,,,,,,,,,,
1230,66.0,NaN,159.0,646.0,NaN,NaN,NaN,NaN,NaN,18.0,NaN
1236,21.0,NaN,84.0,99.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1238,NaN,NaN,45.0,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN
1248,NaN,NaN,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1249,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [9]:
customer_item_matrix=customer_item_matrix.applymap(lambda x:1 if x>0 else 0)

In [10]:
customer_item_matrix.head()

HSNCode,9109100,13019013,19059040,20019000,20049000,20079990,20093900,21033000,21039090,21069099,22090020
Ccode,,,,,,,,,,,
1230,1,0,1,1,0,0,0,0,0,1,0
1236,1,0,1,1,0,0,0,0,0,1,0
1238,0,0,1,0,0,0,0,0,0,1,0
1248,0,0,1,0,0,0,0,0,0,0,0
1249,0,0,0,0,0,0,0,0,0,0,0


In [11]:
customer_item_matrix.shape

(36, 11)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
user_sim_matrix=pd.DataFrame(cosine_similarity(customer_item_matrix))

In [14]:
user_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.000000,1.000000,0.707107,0.500000,0.0,0.0,0.707107,0.577350,0.707107,0.866025,...,0.707107,1.000000,0.894427,0.894427,0.500000,0.816497,0.577350,0.500000,0.707107,0.577350
1,1.000000,1.000000,0.707107,0.500000,0.0,0.0,0.707107,0.577350,0.707107,0.866025,...,0.707107,1.000000,0.894427,0.894427,0.500000,0.816497,0.577350,0.500000,0.707107,0.577350
2,0.707107,0.707107,1.000000,0.707107,0.0,0.0,0.500000,0.408248,0.000000,0.408248,...,0.500000,0.707107,0.632456,0.632456,0.707107,0.577350,0.408248,0.707107,0.500000,0.408248
3,0.500000,0.500000,0.707107,1.000000,0.0,0.0,0.707107,0.577350,0.000000,0.577350,...,0.707107,0.500000,0.447214,0.447214,1.000000,0.408248,0.577350,1.000000,0.707107,0.577350
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
user_sim_matrix.shape

(36, 36)

In [16]:
user_sim_matrix.columns=customer_item_matrix.index

In [17]:
user_sim_matrix['Ccode']=customer_item_matrix.index

In [18]:
user_sim_matrix.head()

Ccode,1230,1236,1238,1248,1249,1254,1256,1259,1262,1264,...,1327,1329,1332,1333,1345,1624,1792,1923,4352,Ccode
0,1.000000,1.000000,0.707107,0.500000,0.0,0.0,0.707107,0.577350,0.707107,0.866025,...,1.000000,0.894427,0.894427,0.500000,0.816497,0.577350,0.500000,0.707107,0.577350,1230
1,1.000000,1.000000,0.707107,0.500000,0.0,0.0,0.707107,0.577350,0.707107,0.866025,...,1.000000,0.894427,0.894427,0.500000,0.816497,0.577350,0.500000,0.707107,0.577350,1236
2,0.707107,0.707107,1.000000,0.707107,0.0,0.0,0.500000,0.408248,0.000000,0.408248,...,0.707107,0.632456,0.632456,0.707107,0.577350,0.408248,0.707107,0.500000,0.408248,1238
3,0.500000,0.500000,0.707107,1.000000,0.0,0.0,0.707107,0.577350,0.000000,0.577350,...,0.500000,0.447214,0.447214,1.000000,0.408248,0.577350,1.000000,0.707107,0.577350,1248
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1249


In [19]:
user_sim_matrix=user_sim_matrix.set_index('Ccode')

In [20]:
user_sim_matrix.head()

Ccode,1230,1236,1238,1248,1249,1254,1256,1259,1262,1264,...,1326,1327,1329,1332,1333,1345,1624,1792,1923,4352
Ccode,,,,,,,,,,,,,,,,,,,,,
1230,1.000000,1.000000,0.707107,0.500000,0.0,0.0,0.707107,0.577350,0.707107,0.866025,...,0.707107,1.000000,0.894427,0.894427,0.500000,0.816497,0.577350,0.500000,0.707107,0.577350
1236,1.000000,1.000000,0.707107,0.500000,0.0,0.0,0.707107,0.577350,0.707107,0.866025,...,0.707107,1.000000,0.894427,0.894427,0.500000,0.816497,0.577350,0.500000,0.707107,0.577350
1238,0.707107,0.707107,1.000000,0.707107,0.0,0.0,0.500000,0.408248,0.000000,0.408248,...,0.500000,0.707107,0.632456,0.632456,0.707107,0.577350,0.408248,0.707107,0.500000,0.408248
1248,0.500000,0.500000,0.707107,1.000000,0.0,0.0,0.707107,0.577350,0.000000,0.577350,...,0.707107,0.500000,0.447214,0.447214,1.000000,0.408248,0.577350,1.000000,0.707107,0.577350
1249,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
user_sim_matrix.loc[1305].sort_values(ascending=False)

Ccode
1291    1.000000
1305    1.000000
1230    0.755929
1236    0.755929
1284    0.755929
1287    0.755929
1327    0.755929
1323    0.676123
1292    0.676123
1314    0.676123
1298    0.676123
1329    0.676123
1332    0.676123
1264    0.654654
1319    0.654654
1297    0.654654
1322    0.617213
1345    0.617213
1238    0.534522
1326    0.534522
1303    0.534522
1923    0.534522
1256    0.534522
1262    0.534522
1259    0.436436
4352    0.436436
1624    0.436436
1320    0.436436
1293    0.436436
1333    0.377964
1254    0.377964
1248    0.377964
1316    0.377964
1792    0.377964
1289    0.377964
1249    0.000000
Name: 1305, dtype: float64

In [22]:
items_boughtby_Vizag=set(customer_item_matrix.loc[1305].iloc[customer_item_matrix.loc[1305].to_numpy()
                                                                     .nonzero()].index)

In [23]:
items_boughtby_Vizag

{9109100, 19059040, 20019000, 20079990, 20093900, 21033000, 21069099}

In [88]:
## ### items bough by vijaywad which are bought by vizag  --b similarity
items_boughtby_ranga=set(customer_item_matrix.loc[1289].iloc[customer_item_matrix.loc[1289].to_numpy()
                                                                     .nonzero()].index)


In [89]:
items_boughtby_ranga

{20019000}

In [26]:
items_torecommendto_ranga=items_boughtby_Vizag-items_boughtby_ranga

In [27]:
items_torecommendto_ranga

{9109100, 20079990, 20093900, 21033000, 21069099}

In [92]:
sales.loc[sales['HSNCode'].isin(items_torecommendto_ranga),['HSNCode','ProductDescription']].drop_duplicates().set_index('HSNCode').head(1)

,ProductDescription
HSNCode,
21069099,IS-86585-L - POHA INSTANT MIX - 65 X 160 GP -CBB


In [95]:
most_similar_cust=user_sim_matrix.loc[1289].sort_values(ascending=False).reset_index().iloc[1,0]

In [97]:
###trial
items_bought_by_4352= set(customer_item_matrix.loc[1289].iloc[customer_item_matrix.loc[1289].to_numpy().nonzero()].index)

In [98]:
items_bought_by_4352     ###actuals of 1323

{20019000}

In [96]:
most_similar_cust

1923

In [86]:
def get_items_to_recommend_cust(cust_a):
    '''returns items to recommend to cust using cust similarities'''
    most_similar_cust=user_sim_matrix.loc[cust_a].sort_values(ascending=False).reset_index().iloc[1,0]
    items_bought_by_cust_a= set(customer_item_matrix.loc[cust_a].iloc[customer_item_matrix.loc[cust_a].to_numpy().nonzero()].index)
    items_bought_by_cust_b=set(customer_item_matrix.loc[most_similar_cust].iloc[customer_item_matrix.loc[most_similar_cust].to_numpy().nonzero()].index)
    items_to_recommend_to_a=items_bought_by_cust_b - items_bought_by_cust_a
    items_description=sales.loc[sales['HSNCode'].isin(items_to_recommend_to_a),['HSNCode','ProductDescription']].drop_duplicates().set_index('HSNCode')
    return items_description

In [87]:
get_items_to_recommend_cust(1289)

,ProductDescription
HSNCode,
19059040,RICE PAPAD AJWAIN - 32 X 75 GP - CBB (MR)
19059040,RICE PAPAD JEERA - 32 X 75 GP - CBB (MR)
19059040,"CHANA MASALA PAPAD (7"") - 80 X 200 GP - CBB (MR)"
19059040,"GARLIC GREEN CHILLI PAPAD 5"" - 80 X 200 GP - ..."
19059040,"UDAD PAPAD 5"" - 80 X 200 GP - CBB (MR)"
19059040,POTATO PAPAD - 32 X 70 GP - CBB (MR)
19059040,SABUDANA PAPAD - 32 X 70 GP - CBB (MR)
19059040,"PUNJABI MASALA PAPAD 5"" - 80 X 200 GP - CBB (MR)"
19059040,"APPALAM 4"" - 144 X 100 GP - CBB (MR)"
